In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [34]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


In [35]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [123]:
train_data = train_dataset.data.to(device).float() / 255.0
train_targets = train_dataset.targets.to(device)

test_data = test_dataset.data.to(device).float() / 255.0
test_targets = test_dataset.targets.to(device)

train_data = train_data.unsqueeze(1)
test_data = test_data.unsqueeze(1)

def get_batches(data, targets, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size], targets[i:i + batch_size]

batch_size = 500
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [129]:
class SimpleConcatRNNCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.fc = nn.Linear(input_size + hidden_size, hidden_size + 18 + 1)

    def forward(self, x_t, h_prev):
        combined = torch.cat([x_t, h_prev], dim=1)
        h_t = F.tanh(self.fc(combined))
        return h_t[:, :5], h_t[:, 5:5+18], h_t[:, 5+18]

In [ ]:
class ExperimentalModel(nn.Module):
    def __init__(self, input_size=784, hidden_size=5):
        super().__init__()
        self.rnn_cell = SimpleConcatRNNCell(input_size, hidden_size)

        self.x0 = nn.Parameter(torch.zeros(1, 28, 28))
        self.h0 = nn.Parameter(torch.randn(1, hidden_size))

        self.fc = nn.Linear(28 * 28, 10)

    def forward(self, x): # (B, 1, 28, 28)
        B, H, W = x.size(0), x.size(2), x.size(3)

        x_t = self.x0.expand(B, 1, H, W)
        h_t = self.h0.expand(B, -1)

        for _ in range(4):
            x_t_flat = x_t.view(B, -1)
            h_t, weight, bias = self.rnn_cell(x_t_flat, torch.zeros_like(h_t))

            x_cat = torch.cat([x_t, x], dim=1)
            weight = weight.view(B, 1, 2, 3, 3)

            x_cat = x_cat.view(1, B * 2, H, W)
            weight = weight.view(B, 2, 3, 3)

            x_t = F.conv2d(x_cat, weight, bias=bias, padding=1, stride=1, groups=B)
            x_t = x_t.view(B, 1, H, W)

        return self.fc(x_t.view(B, -1))

In [ ]:
learning_rate = 0.001 * 1
epochs = 10000

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = ExperimentalModel().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for name, param in model.named_parameters():
    print(f"{name}: {param.numel()} params, requires_grad={param.requires_grad}")

total_params = sum(p.numel() for p in model.parameters())
print()
print(total_params)

x0: 784 params, requires_grad=True
h0: 5 params, requires_grad=True
rnn_cell.fc.weight: 18936 params, requires_grad=True
rnn_cell.fc.bias: 24 params, requires_grad=True
fc.weight: 7840 params, requires_grad=True
fc.bias: 10 params, requires_grad=True

27599


In [133]:
patience = 40
best_val_loss = float('inf')
no_improvement_epochs = 0

all_outputs = []

for epoch in range(10000):
    model.train()
    running_loss = 0.0
    num_batches = 0
    
    for data, target in get_batches(train_data, train_targets, batch_size):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()

        optimizer.step()
        running_loss += loss.item()
        num_batches += 1

    print(f"Epoch [{epoch + 1}/{epochs}], Training Loss: {running_loss / num_batches:.4f}")

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    num_batches = 0
    epoch_outputs = []

    with torch.no_grad():
        for data, target in get_batches(test_data, test_targets, batch_size):
            outputs = model(data)
            loss = criterion(outputs, target)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += target.size(0)
            num_batches += 1
            correct += (predicted == target).sum().item()

            epoch_outputs.append(outputs)

    all_outputs_tensor = torch.cat(epoch_outputs, dim=0)
    all_outputs.append(all_outputs_tensor)

    max_val = torch.max(all_outputs_tensor).item()
    min_val = torch.min(all_outputs_tensor).item()
    median_val = torch.median(all_outputs_tensor).item()
    mean_val = torch.mean(all_outputs_tensor).item()

    accuracy = 100 * correct / total
    val_loss /= num_batches
    print(f"Epoch [{epoch + 1}/{epochs}], Validation Loss: {val_loss:.4f}, Validation Accuracy: {accuracy:.2f}%")
    print(f"Output Summary: Max={max_val:.4f}, Min={min_val:.4f}, Median={median_val:.4f}, Mean={mean_val:.4f}")
    print()

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        no_improvement_epochs = 0
    else:
        no_improvement_epochs += 1

    if no_improvement_epochs >= patience:
        print(f"Early stopping triggered after {epoch + 1} epochs.")
        print(best_val_loss)
        break

Epoch [1/1000], Training Loss: 0.6865
Epoch [1/1000], Validation Loss: 0.2616, Validation Accuracy: 92.33%
Output Summary: Max=30.9226, Min=-30.8429, Median=-0.2518, Mean=-0.1578

Epoch [2/1000], Training Loss: 0.2617
Epoch [2/1000], Validation Loss: 0.2114, Validation Accuracy: 94.03%
Output Summary: Max=32.3505, Min=-28.7160, Median=-0.2892, Mean=-0.3914

Epoch [3/1000], Training Loss: 0.2146
Epoch [3/1000], Validation Loss: 0.2080, Validation Accuracy: 94.21%
Output Summary: Max=39.9824, Min=-38.4827, Median=-0.2550, Mean=-0.3853

Epoch [4/1000], Training Loss: 0.1920
Epoch [4/1000], Validation Loss: 0.1862, Validation Accuracy: 94.66%
Output Summary: Max=37.9198, Min=-43.8055, Median=-0.2580, Mean=-0.4008

Epoch [5/1000], Training Loss: 0.1784
Epoch [5/1000], Validation Loss: 0.1769, Validation Accuracy: 94.84%
Output Summary: Max=40.2884, Min=-43.3893, Median=-0.3838, Mean=-0.5492

Epoch [6/1000], Training Loss: 0.1670
Epoch [6/1000], Validation Loss: 0.1741, Validation Accuracy: 

KeyboardInterrupt: 

In [94]:
for name, param in model.named_parameters():
    print(name, param)

x0 Parameter containing:
tensor([[[-4.8991e-03,  2.5201e-02, -1.1596e-02,  9.7011e-04,  8.4435e-03,
           5.4457e-03,  1.2147e-02,  2.1471e-02,  7.7073e-03, -3.0992e-03,
          -1.3756e-02,  2.0155e-02,  1.1545e-02, -9.1131e-02, -2.3543e-02,
          -2.2076e-02,  7.2532e-03,  1.0363e-02, -1.7747e-02, -1.6975e-02,
          -7.4943e-03, -1.7222e-02, -6.3775e-03,  6.7688e-02,  4.6023e-04,
           3.6882e-03,  8.9947e-03,  1.8488e-02],
         [-5.2839e-03,  8.6376e-04,  3.4281e-02,  2.9808e-03,  3.6648e-03,
           4.5234e-02,  2.3797e-02,  2.6553e-03,  1.0506e-02, -4.4785e-03,
           6.2421e-03,  2.2334e-02, -3.2451e-02,  2.3809e-02, -2.4318e-02,
          -1.3675e-01,  5.2540e-03, -9.0208e-04, -9.9969e-03,  2.3349e-02,
           1.3842e-02, -1.1336e-03,  4.6699e-03,  2.9924e-02,  1.9460e-02,
           3.9729e-03,  4.3285e-02,  4.0165e-02],
         [ 9.5744e-03,  3.5709e-02, -1.2582e-03,  1.4985e-03, -5.3291e-03,
           1.3135e-02,  2.2672e-02,  3.1037e-02, -